In [3]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.tree import DecisionTreeRegressor

In [4]:
df = pd.read_csv("card_cust.csv")
df.head(2)

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,10001,40.900749,0.818182,95.4,0.0,95.4,0.000000,0.166667,0.0,0.083333,0.00,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0
1,10002,3202.467416,0.909091,0.0,0.0,0.0,6442.945483,0.000000,0.0,0.000000,0.25,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0


### 전처리

In [ ]:
df.isna().sum()

In [7]:
df["MINIMUM_PAYMENTS"] = df["MINIMUM_PAYMENTS"].fillna(df["MINIMUM_PAYMENTS"].mean())

In [8]:
df.isna().sum().sum()

0

In [9]:
df_base = df.copy()

### Q1.

In [11]:
df_base["TENURE"].unique()

array([12.,  8., 11.,  9., 10.,  7.,  6.])

In [13]:
df_base_sub = df_base.loc[df_base["TENURE"] == 12, ]
df_base_sub[["BALANCE", "CREDIT_LIMIT"]].corr().iloc[0, 1]

0.4608334883447319

In [21]:
df_corr = df_base.groupby("TENURE")[["BALANCE", "CREDIT_LIMIT"]].corr()
df_corr.round(2)

BALANCE  CREDIT_LIMIT
TENURE                                    
6.0    BALANCE          1.00          0.87
       CREDIT_LIMIT     0.87          1.00
7.0    BALANCE          1.00          0.95
       CREDIT_LIMIT     0.95          1.00
8.0    BALANCE          1.00          0.82
       CREDIT_LIMIT     0.82          1.00
9.0    BALANCE          1.00          0.09
       CREDIT_LIMIT     0.09          1.00
10.0   BALANCE          1.00          0.29
       CREDIT_LIMIT     0.29          1.00
11.0   BALANCE          1.00          0.38
       CREDIT_LIMIT     0.38          1.00
12.0   BALANCE          1.00          0.46
       CREDIT_LIMIT     0.46          1.00

In [22]:
df_corr.iloc[::2, ]

,,BALANCE,CREDIT_LIMIT
TENURE,,,
6.0,BALANCE,1.0,0.868056
7.0,BALANCE,1.0,0.948405
8.0,BALANCE,1.0,0.820696
9.0,BALANCE,1.0,0.085474
10.0,BALANCE,1.0,0.291482
11.0,BALANCE,1.0,0.380360
12.0,BALANCE,1.0,0.460833


In [20]:
df_corr.xs("BALANCE", level = 1)

,BALANCE,CREDIT_LIMIT
TENURE,,
6.0,1.0,0.868056
7.0,1.0,0.948405
8.0,1.0,0.820696
9.0,1.0,0.085474
10.0,1.0,0.291482
11.0,1.0,0.380360
12.0,1.0,0.460833


In [25]:
df_corr.xs("BALANCE", level = 1)["CREDIT_LIMIT"].round(2).max()

0.95

### Q2.

In [26]:
df_base.head(1)

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,10001,40.900749,0.818182,95.4,0.0,95.4,0.0,0.166667,0.0,0.083333,0.0,0.0,2.0,1000.0,201.802084,139.509787,0.0,12.0


In [ ]:
df_q2 = df_base.drop(columns = "CUST_ID")
df_q2.head(1)

In [ ]:
arr_q2_nor = StandardScaler().fit_transform(df_q2)
arr_q2_nor[:1, ]

In [ ]:
ls_k = [2, 3, 4, 5]
k = ls_k[0]

model_kmeans = KMeans(n_clusters = k, random_state = 1234)
model_kmeans.fit(arr_q2_nor)

val_sil = silhouette_score(arr_q2_nor, labels = model_kmeans.labels_)
val_sil

In [ ]:
ls_k = [2, 3, 4, 5]

ls_sil = []
for k in ls_k:
    model_kmeans = KMeans(n_clusters = k, random_state = 1234)
    model_kmeans.fit(arr_q2_nor)

    val_sil = silhouette_score(arr_q2_nor, labels = model_kmeans.labels_)
    ls_sil = ls_sil + [val_sil]

In [ ]:
ser_sil = pd.Series(ls_sil, index = ls_k)
ser_sil

In [ ]:
best_k = ser_sil.idxmax()
best_k

In [ ]:
model_kmeans_b = KMeans(n_clusters = best_k, random_state = 1234)
model_kmeans_b.fit(arr_q2_nor)

In [48]:
df_q2["cluster"] = model_kmeans_b.labels_
df_q2.head(2)

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE,cluster
0,40.900749,0.818182,95.4,0.0,95.4,0.000000,0.166667,0.0,0.083333,0.00,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0,0
1,3202.467416,0.909091,0.0,0.0,0.0,6442.945483,0.000000,0.0,0.000000,0.25,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0,0


In [51]:
df_q2.groupby("cluster")["ONEOFF_PURCHASES"].mean().round(2).max()

3946.19

In [52]:
df_q2["cluster"].value_counts()

cluster
0    802
1    198
Name: count, dtype: int64

### Q3.

In [54]:
df_train = df_base.loc[(df_base["CUST_ID"] % 4) != 0, ]
df_test  = df_base.loc[(df_base["CUST_ID"] % 4) == 0, ]
len(df_train), len(df_test)

(752, 248)

In [64]:
col_drop = ["CUST_ID", "ONEOFF_PURCHASES"]

model_dt = DecisionTreeRegressor(random_state = 1234)
model_dt.fit(X = df_train.drop(columns = col_drop),
             y = df_train["ONEOFF_PURCHASES"])
pred = model_dt.predict(df_test.drop(columns = col_drop))
pred[:4]

array([1500.,    0., 1490.,    0.])

In [62]:
y_t = df_test["ONEOFF_PURCHASES"]
y_p = pred

In [65]:
from sklearn.metrics import mean_squared_error

In [ ]:
round(mean_squared_error(y_true = y_t, y_pred = y_p) ** 0.5, 1)

In [71]:
# y_t - y_p # E, Error
# (y_t - y_p) ** 2 # SE, Squared Error
# ((y_t - y_p) ** 2).mean() # MSE, Mean Squared Error
((y_t - y_p) ** 2).mean() ** 0.5 # RMSE, Root Mean Squared Error

1039.193967231063

In [72]:
round(((y_t - y_p) ** 2).mean() ** 0.5, 1)

1039.2